# BERT model

In [ ]:
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## Loading the data

In [ ]:
df = pd.read_csv('stock_data.csv')

In [ ]:
df.head()

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1


## 1- Import the data, and process it.
The below code defines a function load_news_data that reads a CSV file containing news and their corresponding sentiments. Itreturns a list of review texts and a list of labels, where 1 represents a positive sentiment, and 0 represents a negative sentiment.

In [ ]:
def load_news_data(data_file):
    df = pd.read_csv(data_file)
    texts = df['Text'].tolist()
    labels = [1 if sentiment == 1 else 0 for sentiment in df['Sentiment'].tolist()]
    return texts, labels

In [ ]:
texts, labels = load_news_data('stock_data.csv')

## 2- Define our model’s parameters
Here, we are going to set up essential parameters for fine-tuning the BERTClassifier, including the BERT model name, number of classes, maximum input sequence length, batch size, number of training epochs, and learning rate, to help the model effectively understand movie reviews and their sentiments.

In [ ]:
bert_model_name = 'bert-base-uncased'
num_classes = 1
max_length = 512
batch_size = 32
num_epochs = 12
learning_rate = 2e-5

## 3- Create a custom dataset class for text classification
Tokenize texts and convert them to tensors suitable for TensorFlow.

In [ ]:
def encode_texts(texts, tokenizer):
    # Tokenize texts and convert them to tensors suitable for TensorFlow
    encoding = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="tf")
    return {key: tf.convert_to_tensor(val) for key, val in encoding.items()}

def tf_dataset(texts, labels, batch_size, tokenizer):
    encoded_texts = encode_texts(texts, tokenizer)
    labels = tf.convert_to_tensor(labels)
    dataset = tf.data.Dataset.from_tensor_slices((encoded_texts, labels))
    dataset = dataset.shuffle(len(texts)).batch(batch_size)
    return dataset

## 4- Loading, and splitting the data

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

## 5- Build our customer BERT classifier
In this step, we aim to create our own custom BERT classifier. The classifier is built on top of the famous BERT model, which is great at understanding text. We will then add a dropout layer to keep things in check and a linear layer to help us classify text.

In [ ]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, num_classes):
        super(TFBertClassifier, self).__init__()
        self.bert = TFBertModel.from_pretrained('bert-base-uncased')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, inputs):
        output = self.bert(inputs)
        pooled_output = output.pooler_output
        pooled_output = self.dropout(pooled_output)
        return self.classifier(pooled_output)

## 6- Initialize tokenizer, dataset, and data loader.

In [ ]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = tf_dataset(train_texts, train_labels, batch_size, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## 7- Now the training phase

In [ ]:
model = TFBertClassifier(num_classes= num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
model.fit(train_dataset, epochs= num_epochs)

Epoch 1/12
145/145 [==============================] - 146s 371ms/step - loss: 0.6023 - accuracy: 0.6626
Epoch 2/12
145/145 [==============================] - 55s 379ms/step - loss: 0.4126 - accuracy: 0.8100
Epoch 3/12
145/145 [==============================] - 55s 381ms/step - loss: 0.2205 - accuracy: 0.9119
Epoch 4/12
145/145 [==============================] - 56s 385ms/step - loss: 0.0964 - accuracy: 0.9652
Epoch 5/12
145/145 [==============================] - 56s 389ms/step - loss: 0.0562 - accuracy: 0.9801
Epoch 6/12
145/145 [==============================] - 57s 393ms/step - loss: 0.0368 - accuracy: 0.9879
Epoch 7/12
145/145 [==============================] - 56s 389ms/step - loss: 0.0309 - accuracy: 0.9888
Epoch 8/12
145/145 [==============================] - 57s 391ms/step - loss: 0.0254 - accuracy: 0.9916
Epoch 9/12
145/145 [==============================] - 57s 393ms/step - loss: 0.0094 - accuracy: 0.9955
Epoch 10/12
145/145 [==============================] - 56s 386ms/step - 

## 8- Save the model

In [ ]:
model.save('BERT')

## 9- Convert the model to tf lite

In [ ]:
# Load the SavedModel
saved_model_dir = 'BERT'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Set the optimization strategy if desired (optional)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('BERT.tflite', 'wb') as f:
    f.write(tflite_model)

## 10- Testing the model on data from the internet
Getting different examples from the internet and trying them all out to check the model

In [ ]:
result = encode_texts("ETFs to Tap on NVIDIA's 10-for-1 Stock Split Retail Frenzy", tokenizer)

In [ ]:
result

{'input_ids': <tf.Tensor: shape=(1, 21), dtype=int32, numpy=
 array([[  101,  3802, 10343,  2000, 11112,  2006,  1050, 17258,  2401,
          1005,  1055,  2184,  1011,  2005,  1011,  1015,  4518,  3975,
          7027, 21517,   102]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(1, 21), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(1, 21), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=int32)>}

In [ ]:
prediction = model.predict(result)

1/1 [==============================] - 0s 44ms/step


In [ ]:
class_label = "Good" if prediction[0][0] > 0.5 else "Not Good"
print(f"Prediction: {class_label} (Confidence: {prediction[0][0] * 100:.2f}%)")

Prediction: Good (Confidence: 100.00%)


In [ ]:
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='BERT.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details

[{'name': 'serving_default_attention_mask:0',
  'index': 0,
  'shape': array([ 1, 61], dtype=int32),
  'shape_signature': array([-1, 61], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'serving_default_input_ids:0',
  'index': 1,
  'shape': array([ 1, 61], dtype=int32),
  'shape_signature': array([-1, 61], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'serving_default_token_type_ids:0',
  'index': 2,
  'shape': array([ 1, 61], dtype=int32),
  'shape_signature': array([-1, 61], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=f

In [ ]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 1332,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
prediction

array([[0.999954]], dtype=float32)